# 멋진 작사가 만들기

In [ ]:
import glob
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split

## Step 1. 데이터 다운로드



## Step 2. 데이터 읽어오기

In [ ]:
txt_file_path = '/content/drive/MyDrive/AIFFEL/Exp/05/data/*'

txt_list = glob.glob(txt_file_path)

In [ ]:
raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:", raw_corpus[:5])

데이터 크기: 187088
Examples: ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation', "Baby, can't you read the signs? I won't bore you with the details, baby", "I don't even wanna waste your time"]


## Step 3. 데이터 정제

In [ ]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()                   # 소문자로 바꾸고, 양쪽 공백 제거
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)   # 특수문자 양쪽에 공백 추가
    sentence = re.sub(r'[" "]+', " ", sentence)           # 여러 개의 공백은 하나의 공백으로 수정
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 수정
    sentence = sentence.strip()                           # 양쪽 공백 제거
    sentence = '<start> ' + sentence + ' <end>'           # 문장 시작에는 <start>, 끝에는 <end> 추가
    return sentence

In [ ]:
# 정제된 문장을 담을 리스트
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
corpus[:10]

['<start> looking for some education <end>',
 '<start> made my way into the night <end>',
 '<start> all that bullshit conversation <end>',
 '<start> baby , can t you read the signs ? i won t bore you with the details , baby <end>',
 '<start> i don t even wanna waste your time <end>',
 '<start> let s just say that maybe <end>',
 '<start> you could help me ease my mind <end>',
 '<start> i ain t mr . right but if you re looking for fast love <end>',
 '<start> if that s love in your eyes <end>',
 '<start> it s more than enough <end>']



*   토큰의 개수가 15개를 넘어가는 경우 학습 데이터에서 제외하기



In [ ]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,
        filters=' ',
        oov_token="<unk>"
    )

    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, maxlen=15, padding='post')  # 토큰 최대 개수 15개 제한
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2 306  28 ...   0   0   0]
 [  2 221  13 ...   0   0   0]
 [  2  24  17 ...   0   0   0]
 ...
 [  2  23  77 ...   0   0   0]
 [  2  42  26 ...   0   0   0]
 [  2  23  77 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f9717e45c90>


## Step 4. 평가 데이터셋 분리

In [ ]:
src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]  

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, random_state=4, test_size=0.2)

## Step 5. 인공지능 만들기



*    10 Epoch 안에 loss 값이 2.2 이하로 나오는 모델 설계하기



In [ ]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train)

VOCAB_SIZE = tokenizer.num_words + 1

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(256, 14), dtype=tf.int32, name=None), TensorSpec(shape=(256, 14), dtype=tf.int32, name=None))>

In [ ]:
# 모델 생성 (Subclassing API)
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [ ]:
for src_sample, tgt_sample in dataset.take(1): break
# 모델 빌드
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-1.63181336e-04,  1.95889152e-06,  9.15886631e-05, ...,
          4.12018504e-04, -1.21971156e-04, -5.02773328e-05],
        [-9.52694973e-06,  4.46697231e-06,  2.99327017e-04, ...,
          5.25289157e-04, -6.97285577e-04, -2.75562226e-04],
        [-1.14588147e-04,  2.45886855e-04,  2.28644349e-05, ...,
          4.88593709e-04, -9.08439513e-04, -4.44687699e-04],
        ...,
        [ 1.25827733e-03,  1.97178521e-03,  1.48821482e-03, ...,
          2.50645680e-04, -1.70636515e-04,  4.23447811e-04],
        [ 1.15320657e-03,  1.69427041e-03,  1.56848878e-03, ...,
         -9.20374296e-05,  1.53977307e-04,  4.99512302e-04],
        [ 1.35645247e-03,  1.12317037e-03,  1.32821396e-03, ...,
         -3.02157918e-04,  4.47147875e-04,  5.41375601e-04]],

       [[-2.59502558e-05,  6.39617356e-05,  4.27165389e-04, ...,
         -2.66848765e-05, -2.41005328e-04, -2.11982187e-05],
        [ 1.75172449e-04,  1.72862667e-04,  2

In [35]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

# 모델 컴파일 및 학습
model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
549/549 [==============================] - 506s 916ms/step - loss: 3.6614
Epoch 2/10
549/549 [==============================] - 487s 886ms/step - loss: 3.1549
Epoch 3/10
549/549 [==============================] - 487s 886ms/step - loss: 2.9582
Epoch 4/10
549/549 [==============================] - 488s 889ms/step - loss: 2.8124
Epoch 5/10
549/549 [==============================] - 489s 891ms/step - loss: 2.6852
Epoch 6/10
549/549 [==============================] - 487s 886ms/step - loss: 2.5666
Epoch 7/10
549/549 [==============================] - 474s 863ms/step - loss: 2.4550
Epoch 8/10
549/549 [==============================] - 495s 901ms/step - loss: 2.3495
Epoch 9/10
549/549 [==============================] - 494s 899ms/step - loss: 2.2489
Epoch 10/10
549/549 [==============================] - 483s 879ms/step - loss: 2.1514


In [36]:
# 문장 생성기
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        predict = model(test_tensor) 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]


        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [37]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '



---



# 회고

'언어 모델'이란 개념의 이해부터 시작해 자연어 데이터에 대한 전처리 과정, Subclassing 방식의 모델링, 최종 학습된 내용을 바탕으로 새 문장을 생성하기까지 NLP의 주요 프로세스를 실습해 볼 수 있어서 좋았다.

그러나 아직 전처리 과정에 사용되는 regex 문법이 미숙하고, 모델 파라미터와 Embedding layer에 대한 이해도 완벽하지 못하다.

현재는 10 epoch 내 loss 값이 *2.15* 로 나타나는 데 이를 효과적으로 더 낮추기 위해서 embedding size나 hidden size 외에 또 어느 부분을 수정해 볼 수 있는 지 궁금하다.